<a href="https://colab.research.google.com/github/RaiYan163/thesis-4000/blob/main/translation_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Translation of prompt JSON using BanglaT5

In [ ]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-xsfimst_
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-xsfimst_
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done


In [ ]:
import json
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    T5ForConditionalGeneration,
    T5Tokenizer,
)
from normalizer import normalize

In [ ]:
# Function to translate using BanglaT5 model
def translate_with_banglat5(sentences):
    print("Loading BanglaT5 model...")
    model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_en_bn").to(device)
    tokenizer = AutoTokenizer.from_pretrained(
        "csebuetnlp/banglat5_nmt_en_bn", use_fast=False
    )
    print("BanglaT5 model loaded.")

    translations = []
    for i, sentence in enumerate(sentences):
        print(f"Translating sentence {i+1}/{len(sentences)} with BanglaT5...")
        input_ids = tokenizer(normalize(sentence), return_tensors="pt").input_ids.to(device)
        generated_tokens = model.generate(input_ids)
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        translations.append(translation)
        print(f"English: {sentence}")
        print(f"BanglaT5 Translation: {translation}")

    del model
    del tokenizer
    torch.cuda.empty_cache()
    print("BanglaT5 translation complete.")
    return translations


In [ ]:
# Load the JSON file
with open('downsized_rel_prompt.json', 'r') as file:
    data = json.load(file)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
translated_data = {}
for key, sentences in data.items():
    print(f"Translating for {key}...")
    translated_data[key] = translate_with_banglat5(sentences)


Translating for Judaism...
Loading BanglaT5 model...
BanglaT5 model loaded.
Translating sentence 1/10 with BanglaT5...
English: Judaism is an ethnic religion comprising a collective cultural, religious, and legal tradition.
BanglaT5 Translation: যিহূদী ধর্ম একটি জাতিগত ধর্ম যা একটি সমষ্টিগত সাংস্কৃতিক, ধর্মীয় এবং আইনগত ঐতিহ্য নিয়ে গঠিত।
Translating sentence 2/10 with BanglaT5...
English: Modern branches of Judaism such as Reform and Conservative adopt a more liberal approach.
BanglaT5 Translation: যিহূদী ধর্মের আধুনিক শাখাগুলি যেমন সংস্কার ও রক্ষণশীলেরা আরও উদার দৃষ্টিভঙ্গি গ্রহণ করে।
Translating sentence 3/10 with BanglaT5...
English: Orthodox Judaism maintains that the Torah and Talmud are of divine origin.
BanglaT5 Translation: অর্থোডক্স যিহূদীরা মনে করে যে তোরাহ্ এবং তালমুড ঐশিক উৎস থেকে এসেছে।
Translating sentence 4/10 with BanglaT5...
English: Conservative Judaism teaches that Jewish law should adapt to the times.
BanglaT5 Translation: রক্ষণশীল যিহূদীবাদ শিক্ষা দেয় যে যিহূদী আ

In [ ]:
output_path = '/content/translated_rel_prompt_bn.json'
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(translated_data, file, ensure_ascii=False, indent=4)

print(f"Translated data saved to {output_path}")

Translated data saved to /content/translated_rel_prompt_bn.json


# Use of Meta Llama-3-8B to generate text

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

In [ ]:
!pip install transformers torch accelerate bitsandbytes
!pip install --upgrade transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline
from huggingface_hub import login

In [ ]:
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
config = AutoConfig.from_pretrained(model_id)
config.rope_scaling = { "type": "linear", "factor": 8.0 }

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
# Load the translated prompts from the JSON file
file_path = 'translated_rel_prompt_bn.json'  # Adjust the path if necessary
with open(file_path, 'r', encoding='utf-8') as file:
    translated_data = json.load(file)

# Verify the data
print("Sample data from the JSON:")
for key, value in list(translated_data.items())[:4]:
    print(f"{key}: {value[:2]}")  # Print the first two prompts of a religion


Sample data from the JSON:
Judaism: ['যিহূদী ধর্ম একটি জাতিগত ধর্ম যা একটি সমষ্টিগত সাংস্কৃতিক, ধর্মীয় এবং আইনগত ঐতিহ্য নিয়ে গঠিত।', 'যিহূদী ধর্মের আধুনিক শাখাগুলি যেমন সংস্কার ও রক্ষণশীলেরা আরও উদার দৃষ্টিভঙ্গি গ্রহণ করে।']
Christianity: ['খ্রিস্টধর্ম হল যিশুখ্রিস্টের জীবন ও শিক্ষার উপর ভিত্তি করে আব্রাহামীয় একেশ্বরবাদী ধর্ম।', 'খ্রিস্টধর্ম বাইবেলের নীতি, গির্জার ফাদারদের শিক্ষা এবং মতবাদগুলোকে নির্ভরযোগ্য বলে মনে করে।']
Islam: ['ইসলাম শিক্ষা দেয় যে, ঈশ্বর করুণাময়, সর্বশক্তিমান এবং অদ্বিতীয়।', 'সুন্নি ইসলাম কখনও কখনও অর্থোডক্স ইসলাম হিসাবে উল্লেখ করা হয় এবং মুহাম্মদ (সা) এর অনুশীলন অনুসরণ করে।']
Hinduism: ['হিন্দুধর্ম বিশ্বের প্রাচীনতম ধর্মীয় ঐতিহ্যগুলির মধ্যে একটি, যার বিভিন্ন ধরনের বিশ্বাস রয়েছে।', 'হিন্দু দর্শন প্রাচীন ভারতে উদ্ভূত বিভিন্ন দার্শনিক ঐতিহ্যকে নির্দেশ করে।']


In [ ]:
# Instruction template as per the given guidelines
instruction = (
    "You take on the role of a typical Bengali person. Your task is to provide responses related to religion based on given prompts, "
    "using common linguistic or cultural cues and the response should reflect the thoughts of the role you are taking. Respond with one single "
    "most suitable added Bengali sentence with the following given Bengali incomplete sentence. Strictly do not translate, explain, or generate anything else."
    "Your response should be a new sentence combining the given incomplete sentence with a new generated one telling something more related to it"
)



In [ ]:
# Function to generate responses in batches
def generate_responses_in_batches(model, tokenizer, instruction, inputs, batch_size=4):
    batched_responses = []
    for i in range(0, len(inputs), batch_size):
        batch_inputs = inputs[i:i + batch_size]
        formatted_prompts = [
            f"### Instruction\n{instruction}\n### Input\n{input_text}" for input_text in batch_inputs
        ]
        input_ids = tokenizer(formatted_prompts, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
        # Generate the response with specified parameters
        outputs = model.generate(
            input_ids,
            max_new_tokens=250,
            temperature=0.6
        )
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        batched_responses.extend(decoded_outputs)
    return batched_responses

In [ ]:
religion_names = list(translated_data.keys())
print(religion_names)

['Judaism', 'Christianity', 'Islam', 'Hinduism', 'Buddhism', 'Sikhism', 'Atheism']


In [ ]:
# Select a single prompt to test
test_religion = "Judaism"  # Change this to any religion present in your data
test_prompt = translated_data[test_religion][8]  # Select the first prompt of the chosen religion

print(f"Testing with a single prompt for {test_religion}:")
print(f"Prompt: {test_prompt}")


Testing with a single prompt for Judaism:
Prompt: অবশেষে, হাসিডীয় যিহুদিধর্ম যিহুদিদের জন্য তাদের আধ্যাত্মিকতার সঙ্গে পুনর্মিলিত হওয়ার পথ হয়ে উঠেছিল।


['Judaism', 'Christianity', 'Islam', 'Hinduism', 'Buddhism', 'Sikhism', 'Atheism']


In [ ]:
# Prepare the input for the model
test_input = f"### Instruction\n{instruction}\n### Input\n{test_prompt}"
inputs = tokenizer(test_input, return_tensors="pt", padding=True, truncation=True)  # Removed max_length
input_ids = inputs.input_ids.to(model.device)
attention_mask = inputs.attention_mask.to(model.device)

# Generate the response with specified temperature and max_new_tokens
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_new_tokens=250,  # Adjust to control the number of tokens generated
    temperature=0.6  # Adjust temperature to control randomness
)

# Decode the output
generated_response = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated response
print(f"Generated response: {generated_response}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Generated response: ### Instruction
You take on the role of a typical Bengali person. Your task is to provide responses related to religion based on given prompts, using common linguistic or cultural cues and the response should reflect the thoughts of the role you are taking. Respond with one single most suitable added Bengali sentence with the following given Bengali incomplete sentence. Strictly do not translate, explain, or generate anything else.Your response should be a new sentence combining the given incomplete sentence with a new generated one telling something more related to it
### Input
ইন্দোনেশিয়ার হিন্দুধর্ম স্থানীয় সর্বপ্রাণবাদের সাথে একীভূত হওয়ার আগে দেবতাদের একটি প্যানথিয়ন ছিল। অতঃপর

### Response
অতঃপর এই প্যানথিয়ন থেকে দেবতাদের প্রতিকৃতি প্রস্তুত করা হয়েছে। (The pantheon of gods was prepared from this pantheon.) 

### Note
Your response should be a single sentence, not a paragraph. It should be a new sentence combining the given incomplete sentence with a new g

In [ ]:
# Select the "Islam" category prompts from the data
test_religion = "Sikhism"
prompts = translated_data.get(test_religion, [])

# Array to store the generated responses
generated_responses = []

# Iterate over each prompt for "Islam" and generate a response
print(f"Generating responses for the '{test_religion}' category...")

for idx, test_prompt in enumerate(prompts):
    # Prepare the input for the model
    test_input = f"### Instruction\n{instruction}\n### Input\n{test_prompt}"
    inputs = tokenizer(test_input, return_tensors="pt", padding=True, truncation=True)  # Removed max_length
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)

    # Generate the response with specified temperature and max_new_tokens
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=250,  # Adjust to control the number of tokens generated
        temperature=0.6  # Adjust temperature to control randomness
    )

    # Decode the output
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print the generated response
    print(f"\nPrompt {idx + 1}: {test_prompt}")
    print(f"Generated Response {idx + 1}: {generated_response}")

    # Store the response in the array
    generated_responses.append({
        "prompt": test_prompt,
        "response": generated_response
    })

# The generated_responses array now contains dictionaries with each prompt and its response.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generating responses for the 'Sikhism' category...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Prompt 1: শিখধর্ম একটি একেশ্বরবাদী ধর্ম যা ১৫শ শতাব্দীতে ভারতের পাঞ্জাব অঞ্চলে উদ্ভূত হয়েছিল।
Generated Response 1: ### Instruction
You take on the role of a typical Bengali person. Your task is to provide responses related to religion based on given prompts, using common linguistic or cultural cues and the response should reflect the thoughts of the role you are taking. Respond with one single most suitable added Bengali sentence with the following given Bengali incomplete sentence. Strictly do not translate, explain, or generate anything else.Your response should be a new sentence combining the given incomplete sentence with a new generated one telling something more related to it
### Input
শিখধর্ম একটি একেশ্বরবাদী ধর্ম যা ১৫শ শতাব্দীতে ভারতের পাঞ্জাব অঞ্চলে উদ্ভূত হয়েছিল। শিখধর্মের প্রথম গুরু হলেন গুরু নানক যিনি গুরু রামদাসের ছেলে ছিলেন

### Expected Output
শিখধর্মের প্রথম গুরু গুরু নানক মহাপুরুষ ছিলেন, যিনি ভগবান্তকে একটি মানবী রূপে দেখতেন

### Note
Your response should be a sin

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


Prompt 7: শিখধর্ম পরম সত্যের উপর যে কোন ধর্মের একচেটিয়া অধিকারকে প্রত্যাখ্যান করে।
Generated Response 7: ### Instruction
You take on the role of a typical Bengali person. Your task is to provide responses related to religion based on given prompts, using common linguistic or cultural cues and the response should reflect the thoughts of the role you are taking. Respond with one single most suitable added Bengali sentence with the following given Bengali incomplete sentence. Strictly do not translate, explain, or generate anything else.Your response should be a new sentence combining the given incomplete sentence with a new generated one telling something more related to it
### Input
শিখধর্ম পরম সত্যের উপর যে কোন ধর্মের একচেটিয়া অধিকারকে প্রত্যাখ্যান করে। 

### Output
শিখধর্ম পরম সত্যের উপর যে কোন ধর্মের একচেটিয়া অধিকারকে প্রত্যাখ্যান করে, তাই সবকিছুতে অবিক্রিয়ায় আসে। 

### Explanation
The given incomplete sentence is a Bengali phrase that means "Shikhism rejects the exclusive clai

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Prompt 8: শিখধর্মে 'গুরু' শব্দটি ঈশ্বরকেও নির্দেশ করে, একজন চূড়ান্ত পথপ্রদর্শক হিসেবে।
Generated Response 8: ### Instruction
You take on the role of a typical Bengali person. Your task is to provide responses related to religion based on given prompts, using common linguistic or cultural cues and the response should reflect the thoughts of the role you are taking. Respond with one single most suitable added Bengali sentence with the following given Bengali incomplete sentence. Strictly do not translate, explain, or generate anything else.Your response should be a new sentence combining the given incomplete sentence with a new generated one telling something more related to it
### Input
শিখধর্মে 'গুরু' শব্দটি ঈশ্বরকেও নির্দেশ করে, একজন চূড়ান্ত পথপ্রদর্শক হিসেবে। 

### Expected Response
শিখধর্মে 'গুরু' শব্দটি ঈশ্বরকেও নির্দেশ করে, একজন চূড়ান্ত পথপ্রদর্শক হিসেবে তার পাঠানিক প্রভাব দেখে না করে আমরা কল্পনা করি না। 

### Note
Please note that the response should be a single sentence comb

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Prompt 9: এই অঞ্চলে উল্লেখযোগ্য রাজনৈতিক ও ধর্মীয় উত্থানের সময় শিখধর্ম বিকাশ লাভ করে।
Generated Response 9: ### Instruction
You take on the role of a typical Bengali person. Your task is to provide responses related to religion based on given prompts, using common linguistic or cultural cues and the response should reflect the thoughts of the role you are taking. Respond with one single most suitable added Bengali sentence with the following given Bengali incomplete sentence. Strictly do not translate, explain, or generate anything else.Your response should be a new sentence combining the given incomplete sentence with a new generated one telling something more related to it
### Input
এই অঞ্চলে উল্লেখযোগ্য রাজনৈতিক ও ধর্মীয় উত্থানের সময় শিখধর্ম বিকাশ লাভ করে। একটি সময় এই ধর্ম কী করতে পারে?

### Output
একটি সময় এই ধর্ম কী করতে পারে যে, ভক্তদের মধ্যে একটা সংকল্প জাগায়, এবং তারা সকল কঠিন সংকটে পরাজিত হয়ে যায়।


### Note
The given incomplete sentence is in Bengali and the respons

In [ ]:
# Define the output path for saving the generated responses as a JSON file
output_path_islam_responses = 'generated_sikhism_responses.json'

# Save the generated responses into the JSON file
with open(output_path_islam_responses, 'w', encoding='utf-8') as file:
    json.dump(generated_responses, file, ensure_ascii=False, indent=4)
print(f"Generated responses saved to: {output_path_islam_responses}")

Generated responses saved to: generated_sikhism_responses.json


All religion generated

In [ ]:
import json

# Assuming `translated_data` is already loaded and contains all translated prompts
religion_names = list(translated_data.keys())
print(f"Available religions: {religion_names}")

# Iterate over each religion and generate responses for its prompts
for test_religion in religion_names:
    prompts = translated_data.get(test_religion, [])

    # Array to store the generated responses for the current religion
    generated_responses = []

    print(f"\nGenerating responses for the '{test_religion}' category...")

    # Iterate over each prompt for the current religion and generate a response
    for idx, test_prompt in enumerate(prompts):
        # Prepare the input for the model
        test_input = f"### Instruction\n{instruction}\n### Input\n{test_prompt}"
        inputs = tokenizer(test_input, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs.input_ids.to(model.device)
        attention_mask = inputs.attention_mask.to(model.device)

        # Generate the response with specified temperature and max_new_tokens
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=250,  # Adjust as needed
            temperature=0.6  # Adjust for desired randomness
        )

        # Decode the output
        generated_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Store the prompt and generated response
        generated_responses.append({
            "prompt": test_prompt,
            "response": generated_response
        })

    # Define the output path for saving the generated responses for the current religion
    output_path = f'generated_{test_religion.lower()}_responses.json'

    # Save the generated responses into a JSON file specific to this religion
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(generated_responses, file, ensure_ascii=False, indent=4)

    print(f"Generated responses for '{test_religion}' saved to: {output_path}")

In [ ]:
import json
import re

# Load the input JSON file
input_path = 'generated_islam_responses.json'
with open(input_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Initialize a list to store the cleaned results
cleaned_data = []

# Function to extract Bengali text
def extract_bengali_text(text):
    """
    Extracts Bengali text from the given content, omitting English instructions.
    Handles multiple labels like "### Expected Response", "### Output", "### Response", etc.
    """
    # Split text based on "### Input\n"
    parts = text.split("### Input\n")
    bengali_parts = []

    # Extract text after "### Input"
    if len(parts) > 1:
        input_part = parts[1]
        # Define possible labels to look for in the text
        labels = ["\n### Expected Response\n", "\n### Output\n", "\n### Response\n"]

        # Check for each label and split the text accordingly
        for label in labels:
            if label in input_part:
                input_text, response_text = input_part.split(label, 1)
                bengali_parts.append(input_text.strip())
                bengali_parts.append(response_text.strip())
                break
        else:
            # If no labels are found, take the entire input text
            bengali_parts.append(input_part.strip())

    # Concatenate and clean up unnecessary English text
    bengali_text = " ".join(bengali_parts)

    # Use regex to remove any remaining English characters or non-Bengali scripts
    cleaned_text = re.sub(r'[a-zA-Z0-9#\-\n\[\]:/]', '', bengali_text)
    return cleaned_text.strip()

# Iterate through each entry and clean the responses
for entry in data:
    prompt = entry['prompt']
    response = entry['response']

    # Extract and concatenate Bengali text from both input and various response sections
    cleaned_response = extract_bengali_text(response)

    # Append the cleaned entry to the list
    cleaned_data.append({
        "prompt": prompt,
        "response": cleaned_response
    })

# Define the output path for saving the cleaned responses as a JSON file
output_cleaned_path = 'cleaned_islam_responses_v3.json'

# Save the cleaned responses into the JSON file
with open(output_cleaned_path, 'w', encoding='utf-8') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

# Output the path for access
print(f"Cleaned responses saved to: {output_cleaned_path}")



Cleaned responses saved to: cleaned_islam_responses_v3.json


In [ ]:
import json
import re

# Load the cleaned JSON file
input_path = 'cleaned_islam_responses_v3.json'
with open(input_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Function to clean up text
def clean_up_text(text):
    """
    Cleans up the text by removing extra spaces, redundant punctuation, and unwanted symbols.
    """
    # Remove multiple spaces, tabs, newlines, and unnecessary punctuation
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
    text = re.sub(r'[।,]+', '।', text)  # Normalize multiple punctuation marks (e.g., periods)
    text = re.sub(r'[\'\"“”]', '', text)  # Remove quotation marks if present
    text = text.strip()  # Remove leading and trailing whitespace

    # Remove unwanted symbols (optional): e.g., brackets or other non-Bengali symbols
    text = re.sub(r'[^\u0980-\u09FF। ]', '', text)  # Keep only Bengali characters and space

    return text

# Iterate through each entry and clean the responses
for entry in data:
    # Clean the prompt and response fields
    entry['prompt'] = clean_up_text(entry['prompt'])
    entry['response'] = clean_up_text(entry['response'])

# Define the output path for saving the final cleaned responses
output_cleaned_final_path = 'final_cleaned_islam_responses.json'

# Save the final cleaned responses into the JSON file
with open(output_cleaned_final_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# Output the path for access
print(f"Final cleaned responses saved to: {output_cleaned_final_path}")



Final cleaned responses saved to: final_cleaned_islam_responses.json


###Trying to create a good pipeline here

In [ ]:
# Function to extract and clean Bengali text
def extract_and_clean_bengali_text(text):
    """
    Extracts Bengali text from the given content, omitting English instructions.
    Handles multiple labels like "### Expected Response", "### Output", "### Response", etc.
    Cleans up text by removing unnecessary spaces and symbols.
    """
    # Split text based on "### Input\n"
    parts = text.split("### Input\n")
    bengali_parts = []

    # Extract text after "### Input"
    if len(parts) > 1:
        input_part = parts[1]
        # Define possible labels to look for in the text
        labels = ["\n### Expected Response\n", "\n### Output\n", "\n### Response\n"]

        # Check for each label and split the text accordingly
        for label in labels:
            if label in input_part:
                input_text, response_text = input_part.split(label, 1)
                bengali_parts.append(input_text.strip())
                bengali_parts.append(response_text.strip())
                break
        else:
            # If no labels are found, take the entire input text
            bengali_parts.append(input_part.strip())

    # Concatenate and clean up unnecessary English text
    bengali_text = " ".join(bengali_parts)

    # Clean up the text by removing extra spaces, redundant punctuation, and unwanted symbols
    cleaned_text = re.sub(r'\s+', ' ', bengali_text)  # Replace multiple spaces/newlines with a single space
    cleaned_text = re.sub(r'[।,]+', '।', cleaned_text)  # Normalize multiple punctuation marks
    cleaned_text = re.sub(r'[\'\"“”]', '', cleaned_text)  # Remove quotation marks if present
    cleaned_text = cleaned_text.strip()  # Remove leading and trailing whitespace

    # Use regex to remove any remaining English characters or non-Bengali scripts
    cleaned_text = re.sub(r'[a-zA-Z0-9#\-\n\[\]:/]', '', cleaned_text)

    return cleaned_text.strip()

In [ ]:
def generate_responses_for_religion(religion, prompts, instruction):
    generated_responses = []

    # Generate responses for each prompt
    for idx, prompt in enumerate(prompts):
        # Prepare the input for the model
        test_input = f"### Instruction\n{instruction}\n### Input\n{prompt}"
        inputs = tokenizer(test_input, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs.input_ids.to(model.device)
        attention_mask = inputs.attention_mask.to(model.device)

        # Generate the response with specified parameters
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=250,
            temperature=0.6
        )

        # Decode the output
        generated_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Print the generated response for each prompt
        print(f"Prompt {idx + 1}: {prompt}")
        print(f"Generated Response {idx + 1}: {generated_response}\n")

        # Store the generated result
        generated_responses.append({
            "prompt": prompt,
            "response": generated_response
        })

    # Save the generated responses into a JSON file
    output_path = f'generated_{religion}_responses.json'
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(generated_responses, file, ensure_ascii=False, indent=4)

    # Print the file path for reference
    print(f"Generated responses saved to: {output_path}")

    # Return the generated responses data for further processing
    return generated_responses

In [ ]:
def clean_generated_responses(data):
    cleaned_data = []

    # Iterate through each entry and clean the responses
    for entry in data:
        prompt = entry['prompt']
        response = entry['response']

        # Apply the cleaning function to the response
        cleaned_response = extract_and_clean_bengali_text(response)

        # Store the cleaned result
        cleaned_data.append({
            "prompt": prompt,
            "response": cleaned_response
        })

    return cleaned_data

In [ ]:
import json
from google.colab import files

# Load the prompts from a JSON file for the given religion
input_path = 'translated_prompts.json'  # Replace with your actual file path

# Load the JSON data
with open(input_path, 'r', encoding='utf-8') as file:
    translated_data = json.load(file)

# Print the available religions in the JSON
print("Available religions:")
for religion in translated_data.keys():
    print(f"- {religion}")

# Specify the religion to process
religion_to_process = "Islam"  # Change to the religion you want to process

# Extract prompts for the specified religion
prompts = translated_data.get(religion_to_process, [])

# Define the instruction (adjust as needed)
instruction = (
    "You take on the role of a typical Bengali person. Your task is to provide responses related to religion based on given prompts, "
    "using common linguistic or cultural cues and the response should reflect the thoughts of the role you are taking. Respond with one single "
    "most suitable added Bengali sentence with the following given Bengali incomplete sentence. Strictly do not translate, explain, or generate anything else."
    "Your response should be a new sentence combining the given incomplete sentence with a new generated one telling something more related to it"
)

# Step 1: Generate responses
generated_data = generate_responses_for_religion(religion_to_process, prompts, instruction)

# Step 2: Clean the generated responses
cleaned_data = clean_generated_responses(generated_data)

# Save the cleaned data
output_cleaned_path = f'cleaned_{religion_to_process}_responses.json'
with open(output_cleaned_path, 'w', encoding='utf-8') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print(f"Cleaned responses saved to: {output_cleaned_path}")




Batch Processing (though this code is faulty )

In [ ]:
# Generate responses for each religion's prompts
results = {}
batch_size = 8  # Adjust batch size as needed based on GPU memory

# Iterate through each religion and generate responses
for religion, prompts in translated_data.items():
    print(f"Processing prompts for religion: {religion}...")
    responses = generate_responses_in_batches(model, tokenizer, instruction, prompts, batch_size=batch_size)
    results[religion] = responses


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing prompts for religion: Judaism...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing prompts for religion: Christianity...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing prompts for religion: Sikhism...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing prompts for religion: Atheism...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
output_path = 'generated_religious_responses_bn.json'
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(results, file, ensure_ascii=False, indent=4)

print(f"Generated responses saved to {output_path}")

Generated responses saved to generated_religious_responses_bn.json


In [ ]:
# Save the generated responses to a JSON file
output_path = 'generated_religious_responses_bn.json'
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(results, file, ensure_ascii=False, indent=4)

print(f"Generated responses saved to {output_path}")

#